In [1]:
import requests
import pandas as pd
import numpy as np
import datetime
import warnings
#Suppress the warning
warnings.filterwarnings("ignore", category=FutureWarning)
from tqdm import tqdm
from time import sleep
import random
from statsbombpy import sb
from pathlib import Path


email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
creds = {"user": email, "passwd": password}

export_path = "/Users/adriaanmartiny/Documents/KAA Gent - JPL Data/Statsbomb/"

def load_comps():
    email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
    password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
    creds = {"user": email, "passwd": password}
    url = 'https://data.statsbomb.com/api/v4/competitions'
    competitions = sb.competitions(creds=creds)

    return competitions.competition_id.unique().tolist(), competitions.season_id.unique().tolist()

def load_season_data(compIds,seasonIds):
    email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
    password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
    creds = {"user": email, "passwd": password}
    seasonStatsList = []
    for c in compIds:
        for s in seasonIds:
            try:
                compInfo = sb.player_season_stats(competition_id=c, season_id=s, creds=creds)
                seasonStatsList.append(compInfo)
                print(f"Seasonstats loaded for {c} in {s}")
            except:
                pass

    loadedStats = pd.concat(seasonStatsList)
    
    return loadedStats

def load_matches(compIds,seasonIds):
    email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
    password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
    creds = {"user": email, "passwd": password}
    gamesList = []
    for c in compIds:
        for s in seasonIds:
            try:
                gameInfo = sb.matches(competition_id=c, season_id=s,creds=creds)
                gamesList.append(gameInfo)
                print(f"Games loaded for {c} in {s}")
            except:
                pass

    loadedGames = pd.concat(gamesList)
    
    return loadedGames


def load_events(matchIds):
    email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
    password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
    creds = {"user": email, "passwd": password}
    eventsList = []
    for m in tqdm(matchIds):
        try:
            eventsInfo = sb.events(match_id=m,creds=creds)
            eventsInfo = eventsInfo.dropna(axis=1, how='all')
            eventsList.append(eventsInfo)
            sleep(random.uniform(0.01, 1.8))
        except:
            pass

    loadedEvents = pd.concat(eventsList)
    
    return loadedEvents




def load_player_match(games):
    email = "adriaan.martiny@kaagent.be"  # email address that you use to login to StatsBomb IQ
    password = "GxuOIhdQ"  # password that you use to login to StatsBomb IQ
    creds = {"user": email, "passwd": password}
    playermatchList = []
    for g in tqdm(games):    
        try:
            playermatchInfo = sb.player_match_stats(g,creds=creds)
            
             # Check if the result is a non-empty DataFrame
            if isinstance(playermatchInfo, pd.DataFrame) and not playermatchInfo.empty:
                playermatchList.append(playermatchInfo)
            else:
                print(f"No data for match_id: {g}. Skipping.")

        except Exception as e:
            tqdm.write(f"[ERROR] Failed to process match_id {g}: {e}")
            continue

            
    loadedplayerMatch = pd.concat(playermatchList)
    
    return loadedplayerMatch


In [2]:
allGames = pd.read_csv(f"{export_path}Games.csv",sep=";")
allGames_a = allGames[allGames.match_status == 'available']
gameslist = allGames_a.match_id.unique().tolist()
print(len(gameslist))

17347


In [3]:
# Group by 'competition' and 'season', then aggregate unique match_ids
batches = (
    allGames_a.groupby(['competition', 'season'])['match_id']
    .unique()
    .reset_index(name='match_ids')
)

# Optional: convert match_ids from numpy array to list
batches['match_ids'] = batches['match_ids'].apply(list)

# Each row in 'batches' is a batch with a unique combination and the corresponding match_ids
print(len(batches))

83


In [4]:
for b in range(len(batches)):
    filename = f'{export_path}/PlayerGame/player_stats_{batches.competition.iloc[b].replace("-","").replace(" ","").replace("/","")}_{batches.season.iloc[b].replace("/","")}.csv'
    
    if Path(filename).exists():
        print("File exists. Skipping step.")
    
    else:
        batch = batches.match_ids.iloc[b]
        batch_matches = load_player_match(batch)
        print(filename)
        batch_matches.to_csv(f"{filename}"
        
        ,index=False,sep=';')

        print(f"Process complete for {batches.competition.iloc[b]} in season {batches.season.iloc[b]}")
        sleep(random.uniform(10,19.3))
    

File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skipping step.
File exists. Skippin

 67%|███████████████████████████▎             | 347/521 [14:58<15:21,  5.30s/it]

https://data.statsbombservices.com/api/v4/matches/3876818/player-stats -> 502
No data for match_id: 3876818. Skipping.


100%|█████████████████████████████████████████| 521/521 [21:59<00:00,  2.53s/it]


/Users/adriaanmartiny/Documents/KAA Gent - JPL Data/Statsbomb//PlayerGame/player_stats_UnitedStatesofAmericaMajorLeagueSoccer_2023.csv
Process complete for United States of America - Major League Soccer in season 2023


100%|█████████████████████████████████████████| 522/522 [23:14<00:00,  2.67s/it]


/Users/adriaanmartiny/Documents/KAA Gent - JPL Data/Statsbomb//PlayerGame/player_stats_UnitedStatesofAmericaMajorLeagueSoccer_2024.csv
Process complete for United States of America - Major League Soccer in season 2024


100%|█████████████████████████████████████████| 207/207 [09:38<00:00,  2.79s/it]


/Users/adriaanmartiny/Documents/KAA Gent - JPL Data/Statsbomb//PlayerGame/player_stats_UnitedStatesofAmericaMajorLeagueSoccer_2025.csv
Process complete for United States of America - Major League Soccer in season 2025
